In [ ]:
import torch
from processing.static_concept_detection import get_structured_engine_evaluation
from processing.concept_generation import analyze_puzzle_concept_importance
from processing.models import Stockfish
import os
import numpy as np
import pandas as pd
from openai import OpenAI
import json
from tqdm import tqdm
from dotenv import load_dotenv

load_dotenv()
api_key = os.getenv("API_KEY")

stockfish_8_path = "Stockfish-8/Windows/stockfish_8_x64.exe"
stockfish_engine = Stockfish(path=stockfish_8_path, parameters={"Threads": 1})

# --- Initialization ---
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


Using PyTorch device: cpu


In [ ]:
# Baseline + Engine
PUZZLE_CSV_PATH = 'subset_puzzles_for_commentary.csv'
OUTPUT_JSONL_PATH = 'commentary_gen/generated_commentaries_baseline_engine.jsonl'
STOCKFISH_PATH = "Stockfish-8/Windows/stockfish_8_x64.exe"
LLM_MODEL = "gpt-4o"

# --- Initialization ---",
client = OpenAI()

base_prompt_zeroshot = [
{
"role": "system",
"content": """
You are an expert chess commentator generating commentary for a club-level player. Explain the short tactical sequence **strictly from the perspective of the player making the `target_move`**.
Your goal is to provide a concise, clear, and fluid explanation focusing **purely on the chess actions, tactics, and consequences on the board.**
Use the board states, the moves provided, and the stockfish evaluation to accomplish this.

**Commentary Generation (Your Actual Output):**
- Write a fluid commentary describing the sequence **only in terms of chess moves, piece interactions, threats, captures, and resulting advantages.**

**Output Format:**
- Output ONLY the `**Commentary:**` section.
"""
}

]

print(f"Loading puzzles from: {PUZZLE_CSV_PATH}")
df_puzzles = pd.read_csv(PUZZLE_CSV_PATH)

print(f"Found {len(df_puzzles)} puzzles.")
generated_count = 0


with open(OUTPUT_JSONL_PATH, 'a') as outfile:
    for index, row in tqdm(df_puzzles.iterrows(), total=len(df_puzzles), desc="Generating Zero-Shot Commentaries"):
        try: 
            puzzle_id = row['PuzzleId']
            initial_fen = row['FEN']
            moves_str = row['Moves']
            puzzle_moves_uci = moves_str.split()

            engine_eval = get_structured_engine_evaluation(
                initial_fen,
                puzzle_moves_uci,
                stockfish_engine,
                depth=20
            )
            if not engine_eval: 
                 print(f"Skipping Puzzle {puzzle_id} due to engine evaluation error.")
                 continue

            llm_payload_zeroshot = {
                "target_fen_s0": engine_eval.get('target_fen'),
                "target_move": engine_eval.get("target_move_dict"),
                "opponent_reply": engine_eval.get("opponent_reply_dict"),
                "target_fen_s2": engine_eval.get('fen_state_2'),
                "best_move_after_s2": engine_eval.get("best_move_after_s2_dict")
            }
            if not all(llm_payload_zeroshot.values()):
                 print(f"Skipping Puzzle {puzzle_id} due to missing data in engine_eval.")
                 continue
            
            current_prompt = base_prompt_zeroshot + [{"role": "user", "content": json.dumps(llm_payload_zeroshot, indent=2)}]

            response = client.chat.completions.create(
                model=LLM_MODEL,
                messages=current_prompt,
                temperature=0.01 
            )
            llm_response_content = response.choices[0].message.content.strip() 
            print(f"Puzzle {puzzle_id}: {llm_response_content}\n") 

            result_record = {
                "puzzle_id": puzzle_id,
                "initial_fen": initial_fen,
                "moves_uci": puzzle_moves_uci,
                "target_move_san": engine_eval.get('target_move_dict'),
                "fen_s0": engine_eval.get('target_fen'),
                "fen_s2": engine_eval.get('fen_state_2'),
                "llm_commentary": llm_response_content 
            }
            outfile.write(json.dumps(result_record) + '\n')
            generated_count += 1

        except Exception as e:
             print(f"\n--- Error processing Puzzle ID {puzzle_id} (Index {index}) ---")
             print(f"FEN: {initial_fen}")
             print(f"Moves: {moves_str}")
             print(f"Error: {e}")
             # Optionally add a delay or specific error handling here
             # time.sleep(1) # Example delay


print(f"\nFinished generation. Generated {generated_count} zero-shot commentaries.")
# --- CHANGE: Update final message ---
print(f"Results saved to: {OUTPUT_JSONL_PATH}")

# --- END OF FILE ---

In [ ]:
# Just baseline
PUZZLE_CSV_PATH = 'subset_puzzles_for_commentary.csv' 
OUTPUT_JSONL_PATH = 'commentary_gen/generated_commentaries_baseline_test.jsonl' 
ONNX_MODEL_PATH = 'prepared_network_fp16.onnx' 
PROBE_SAVE_DIR = 'concepts_sequential/saved_probes' 
STOCKFISH_PATH = "Stockfish-8/Windows/stockfish_8_x64.exe" 

# LLM & Probe Config
LLM_MODEL = "gpt-4o" 

client = OpenAI()

base_prompt = [
{
"role": "system",
"content": """
You are an expert chess commentator generating commentary for a club-level player. Explain the short tactical sequence **strictly from the perspective of the player making the `target_move`**.
Your goal is to provide a concise, clear, and fluid explanation focusing **purely on the chess actions, tactics, and consequences on the board.**
Use the board states, and the moves given to accomplish this.

**Commentary Generation (Your Actual Output):**
- Write a fluid commentary describing the sequence **only in terms of chess moves, piece interactions, threats, captures, and resulting advantages.**

"""
},
]

# --- Main Processing Logic ---
print(f"Loading puzzles from: {PUZZLE_CSV_PATH}")
df_puzzles = pd.read_csv(PUZZLE_CSV_PATH)

print(f"Found {len(df_puzzles)} puzzles.")
generated_count = 0

# Open output file in append mode
with open(OUTPUT_JSONL_PATH, 'a') as outfile:
    for index, row in tqdm(df_puzzles.iterrows(), total=len(df_puzzles), desc="Generating Commentaries"):
        puzzle_id = row['PuzzleId'] 
        initial_fen = row['FEN']
        moves_str = row['Moves']
        puzzle_moves_uci = moves_str.split()

        # 2. Get structured engine evaluation
        engine_eval = get_structured_engine_evaluation(
            initial_fen,
            puzzle_moves_uci,
            stockfish_engine,
            depth=20 
        )

        # 3. Prepare LLM Payload
        llm_payload = {
            "target_fen_s0": engine_eval['target_fen'],
            "target_move": engine_eval["target_move_dict"],
            "opponent_reply": engine_eval["opponent_reply_dict"],
            "target_fen_s2": engine_eval['fen_state_2'],
            "best_move_after_s2": engine_eval["best_move_after_s2_dict"],
        }

        # 4. Construct final prompt
        current_prompt = base_prompt + [{"role": "user", "content": json.dumps(llm_payload, indent=2)}]

        # 5. Call LLM API
        response = client.chat.completions.create(
            model=LLM_MODEL,
            messages=current_prompt,
            temperature=0.01 
        )
        llm_response_content = response.choices[0].message.content
        print(llm_response_content)

        # 6. Parse LLM Response (basic split)
        commentary = llm_response_content 

        # 7. Store Result
        result_record = {
            "puzzle_id": puzzle_id,
            "initial_fen": initial_fen,
            "moves_uci": puzzle_moves_uci,
            "target_move_san": engine_eval['target_move_dict'],
            "fen_s0": engine_eval['target_fen'],
            "best_move_after_s2": engine_eval["best_move_after_s2_dict"],
            "fen_s2": engine_eval['fen_state_2'],
            "llm_commentary": commentary
        }
        outfile.write(json.dumps(result_record) + '\n')
        generated_count += 1



print(f"\nFinished generation. Generated {generated_count} commentaries.")
print(f"Results saved to: {OUTPUT_JSONL_PATH}")
